# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    size= 5,
    color='City',
    frame_width=800,
    frame_height=600
)

# Display the map
map_plot

:Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_filtered = city_data_df[
    (city_data_df['Max Temp'] > 20) &  # Max Temp greater than 20C
    (city_data_df['Humidity'] >= 40) & (city_data_df['Humidity'] <= 90) &  # Humidity between 40% and 90%
    (city_data_df['Wind Speed'] < 4)  # Wind Speed less than 4 kmph
]

# Drop any rows with null values
city_data_df_filtered = city_data_df_filtered.dropna()

# Display sample data
city_data_df_filtered.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,puerto ayora,-0.7393,-90.3518,22.98,82,41,1.79,EC,1666108242
20,20,avarua,-21.2078,-159.7750,22.03,73,20,2.57,CK,1666108220
27,27,bambous virieux,-20.3428,57.7575,22.14,68,75,3.60,MU,1666108246
38,38,labuhan,-6.8844,112.2051,25.24,88,100,2.74,ID,1666108254
39,39,tomatlan,19.9333,-105.2500,28.05,71,93,0.58,MX,1666108255


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_filtered[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
19,puerto ayora,EC,-0.7393,-90.3518,82,
20,avarua,CK,-21.2078,-159.7750,73,
27,bambous virieux,MU,-20.3428,57.7575,68,
38,labuhan,ID,-6.8844,112.2051,88,
39,tomatlan,MX,19.9333,-105.2500,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # We are specifically searching for hotels
    "apiKey": geoapify_key,  # Radius of search in meters
    "limit": 20  # Limit the results to the nearest hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # Use circle to specify search area
    params["bias"] = f"proximity:{longitude},{latitude}"  # Bias the results towards the city location

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
bambous virieux - nearest hotel: Otentic Eco Tent Experience
labuhan - nearest hotel: No hotel found
tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
businga - nearest hotel: Hôtel Roho
lasa - nearest hotel: Hotel "Drei Kreuz"
banda - nearest hotel: #acnindiafy21
cobija - nearest hotel: Hospedaje Novedades
mahebourg - nearest hotel: Grand Bel Air
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
rafaela - nearest hotel: Parra Hotel & Suites
ponta do sol - nearest hotel: Estalagem Ponta do Sol
calabozo - nearest hotel: Gran Hotel Guarico
sao filipe - nearest hotel: Bela Vista
hithadhoo - nearest hotel: Pebbles Inn
bocana de paiwas - nearest hotel: No hotel found
ambon - nearest hotel: Hotel Hero
georgetown - nearest hotel: Page 63 hostel
biak - nearest hotel: 

,City,Country,Lat,Lng,Humidity,Hotel Name
19,puerto ayora,EC,-0.7393,-90.3518,82,Hostal La Mirada De Solitario George
20,avarua,CK,-21.2078,-159.7750,73,Paradise Inn
27,bambous virieux,MU,-20.3428,57.7575,68,Otentic Eco Tent Experience
38,labuhan,ID,-6.8844,112.2051,88,No hotel found
39,tomatlan,MX,19.9333,-105.2500,71,Hotel Hacienda Vieja Tomatlan


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:

# Create interactive map with hotels using Hvplot
hotel_map = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    color='City',
    size=8,
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],
    frame_width=800,
    frame_height=600,
    title='Filtered Cities with Nearby Hotels'
)

# Display the map
hotel_map

:Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)